In [1]:
import pandas as pd
import numpy as np
import janitor
import unidecode
import pickle
import os
import operator
import functools
import itertools
from typing import Callable

In [2]:
if 'raw_databases.pkl' not in os.listdir('./data/'):
    from quickstart.loader import XlsxDriveLoader

    Loads = XlsxDriveLoader()  # Drive Folder is hardcoded in module, since this is not prone to change
    databases = Loads.content  # Process takes approximately ~3 minutes to run for the first time.
                               # Then the file will be stored in the data/
else:
    with open('./data/raw_databases.pkl', 'rb') as file:
        databases = pickle.load(file)
    print("Loaded data from .pkl file")

Loaded data from .pkl file


In [3]:
(db_name_contaminantes, db_name_meteoorologicas, 
db_name_anotaciones, _) = databases.keys()


get_col_names: Callable[[pd.core.frame.DataFrame], pd.core.indexes.base.Index] = (
                         lambda df: sorted(list(df.columns)))

def shared_structure(db_name: str) -> list:
    """Returns sheets in dict of DataFrames that conain the same fields"""

    df_col_names = [[key, get_col_names(val)] for key, val in databases[db_name].items()]

    grouped = list()
    for _, g in itertools.groupby(df_col_names, operator.itemgetter(1)):
        group = list(g) 
        if len(group) > 1: # Can't be performed with comprehension since g is a generator/iterator
            grouped.append(group)
    
    if len(grouped) == 1: 
        return [sheet_name for sheet_name, _ in functools.reduce(operator.iconcat, grouped, [])] 
        # Expecting there is only one shared_structure across the DataFrames
    else:
        return ... # Not necessary

In [4]:
raw_contaminantes = pd.concat([databases[db_name_contaminantes][sheet] 
                            for sheet in shared_structure(db_name_contaminantes)])

raw_meteorologicas = pd.concat([databases[db_name_meteoorologicas][sheet] 
                            for sheet in shared_structure(db_name_meteoorologicas)])

In [5]:
# Dictionary for categorical values

identifier = (
    databases[db_name_anotaciones]['LEEME']
    .loc[:, ['Flag', 'Hora']]
    .set_index('Flag')
    .dropna(axis='index')
    .squeeze()
    .apply(lambda string: unidecode.unidecode(string).strip().lower())
    .apply(lambda validity: True if validity == 'valida' else False)
    .to_dict()
)

identifier |= {'x': False} # Record does not appear on DataFrame


f_identifier = lambda dict_: identifier.get(dict_, True)

In [6]:
# Dictionary for mesaurement units

measurement_units = (
    databases[db_name_anotaciones]['Hoja1'].
    iloc[23:41, [0, 2]]
    .dropna()
    .drop(33)
    .pivot(columns='Notas a considerar:', values='Unnamed: 2')
    .pipe(janitor.clean_names, remove_special=True)
    .mode()
    .squeeze()
    .to_dict()
)

with open('data/measurement_units.pkl', 'wb') as file:
            pickle.dump(measurement_units, file)

In [7]:
fields = ['parametro', 'Fecha',
          'SO2', 'SO2 b']  # San Pedro identifiers


def munge(df: pd.core.frame.DataFrame, fields: list):
    return (
        df
        .loc[:, fields]
        .pipe(janitor.rename_columns, new_column_names={'parametro': 'factor',
                                                        'Fecha': 'date'})
        .pipe(janitor.process_text, column_name='factor', string_function='strip')
        .pivot(index='date', columns='factor', values=['SO2', 'SO2 b'])
        .convert_dtypes(convert_integer=False)
        .apply(lambda field: field.where(field > 0, pd.NA) if pd.api.types.is_numeric_dtype(field)
                                    else field.map(f_identifier))  # Remove negative values from numeric fields
                                                                   # and map str typed variables with a dict
        .pipe(janitor.clean_names, strip_underscores='r')
    )


In [8]:
contaminantes = munge(raw_contaminantes, fields)
meteorologicas = munge(raw_meteorologicas, fields)

In [9]:
# There is a Timestamp missing in df metereologicas
set(contaminantes.index).difference(set(meteorologicas.index))

{Timestamp('2019-05-15 20:00:00')}

In [10]:
df_ = contaminantes.join(other=meteorologicas,
                         how='outer')

In [11]:
df_.describe(include="all").T

count unique   top   freq        mean        std    min  \
      factor                                                              
so2   co      29973.0    NaN   NaN    NaN    1.679246   1.025919   0.01   
      no      31723.0    NaN   NaN    NaN    9.118173   8.916451    0.5   
      no2     32607.0    NaN   NaN    NaN    11.03015     7.8801    0.1   
      nox     32626.0    NaN   NaN    NaN   19.729188  13.361774    0.5   
      o3      37936.0    NaN   NaN    NaN   25.389788  19.092307    1.0   
      pm10    36946.0    NaN   NaN    NaN   60.021545  35.906347    2.0   
      pm2_5   20458.0    NaN   NaN    NaN   18.928187  13.211943    2.0   
      so2     29466.0    NaN   NaN    NaN    4.205708    2.24159    0.5   
so2_b co        39394      2  True  30073         NaN        NaN    NaN   
      no        39394      2  True  31704         NaN        NaN    NaN   
      no2       39394      2  True  32601         NaN        NaN    NaN   
      nox       39394      2  True  32610         NaN        NaN    NaN   
      o3        39394      2  True  38078         NaN        NaN    NaN   
      pm10      39394      2  True  37080         NaN        NaN    NaN   
      pm2_5     39394      2  True  20617         NaN        NaN    NaN   
      so2       39394      2  True  29557         NaN        NaN    NaN   
so2   prs     38538.0    NaN   NaN    NaN  713.374215    1.65867  702.4   
      rainf    1199.0    NaN   NaN    NaN    0.210325   1.385718   0.01   
      rh      38636.0    NaN   NaN    NaN   56.419402  19.152998    1.0   
      sr      20358.0    NaN   NaN    NaN    0.337981   0.268119  0.001   
      tout    38499.0    NaN   NaN    NaN   22.294383   6.885476   0.04   
      wdr     38498.0    NaN   NaN    NaN  113.355681  64.009974    1.0   
      wsr     29532.0    NaN   NaN    NaN    4.867926   6.685375    0.1   
so2_b prs       39393      2  True  39058         NaN        NaN    NaN   
      rainf     39393      2  True  39137         NaN        NaN    NaN   
      rh        39393      2  True  38791         NaN        NaN    NaN   
      sr        39393      2  True  20827         NaN        NaN    NaN   
      tout      39393      2  True  38822         NaN        NaN    NaN   
      wdr       39393      2  True  38718         NaN        NaN    NaN   
      wsr       39393      2  True  29797         NaN        NaN    NaN   

                25%    50%    75%    max  
      factor                              
so2   co        0.8    1.5   2.39   5.79  
      no        3.2    8.2   11.0  161.0  
      no2       5.7    8.8   14.3   82.3  
      nox      12.3   16.9   23.3  198.2  
      o3       10.0   22.0   35.0  153.0  
      pm10     37.0   52.0   75.0  714.0  
      pm2_5     9.0   16.0   25.0  156.0  
      so2       2.9    3.6    4.8   67.4  
so2_b co        NaN    NaN    NaN    NaN  
      no        NaN    NaN    NaN    NaN  
      no2       NaN    NaN    NaN    NaN  
      nox       NaN    NaN    NaN    NaN  
      o3        NaN    NaN    NaN    NaN  
      pm10      NaN    NaN    NaN    NaN  
      pm2_5     NaN    NaN    NaN    NaN  
      so2       NaN    NaN    NaN    NaN  
so2   prs     712.3  713.2  714.3  738.9  
      rainf    0.01   0.02   0.06  17.99  
      rh       42.0   58.0   72.0   92.0  
      sr      0.089  0.291  0.556  0.971  
      tout     17.7  23.12  27.17  40.44  
      wdr      71.0   87.0  122.0  360.0  
      wsr       1.4    4.2    6.4  176.6  
so2_b prs       NaN    NaN    NaN    NaN  
      rainf     NaN    NaN    NaN    NaN  
      rh        NaN    NaN    NaN    NaN  
      sr        NaN    NaN    NaN    NaN  
      tout      NaN    NaN    NaN    NaN  
      wdr       NaN    NaN    NaN    NaN  
      wsr       NaN    NaN    NaN    NaN

In [12]:
df = (
    df_
    .apply(lambda row: row.so2.where(row.so2_b.astype('bool')), axis=1) # check validity
    .replace({pd.NA: np.nan}) # .astype() does not operate when having diferent dtypes for
                              # missing values
    .astype(float)
    .drop_duplicates()
)

# creating missing timestamps, statistical imputation and outlier analysis
# will proceed after data exploration and visualization. Since we are working
# with a time-series database, these proceedures have to be done carefully and
# with precise methodologies.

In [13]:
df

factor,co,no,no2,nox,o3,pm10,pm2_5,so2,prs,rainf,rh,sr,tout,wdr,wsr
date,,,,,,,,,,,,,,,
2017-01-01 00:00:00,1.10,17.0,20.1,37.0,6.0,163.0,NaN,5.0,712.5,NaN,54.0,NaN,20.68,253.0,4.0
2017-01-01 01:00:00,1.05,19.5,25.1,44.5,5.0,117.0,NaN,3.9,712.4,NaN,49.0,NaN,21.59,244.0,5.4
2017-01-01 02:00:00,0.91,16.7,22.5,39.0,5.0,46.0,NaN,3.5,712.3,NaN,51.0,NaN,20.94,260.0,4.4
2017-01-01 03:00:00,0.75,12.5,16.8,29.2,8.0,36.0,NaN,3.4,712.1,NaN,51.0,NaN,20.99,257.0,3.7
2017-01-01 04:00:00,0.59,6.5,6.7,13.1,12.0,25.0,NaN,3.2,711.9,NaN,51.0,NaN,20.89,84.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-30 19:00:00,0.67,4.4,11.3,15.7,17.0,36.0,14.0,3.5,714.1,0.05,80.0,0.029,22.78,69.0,27.1
2021-06-30 20:00:00,0.66,3.9,10.7,14.6,16.0,41.0,7.0,3.6,714.2,NaN,75.0,0.007,23.57,74.0,97.5
2021-06-30 21:00:00,0.64,3.4,11.7,15.1,15.0,16.0,7.0,3.4,714.3,NaN,78.0,NaN,22.89,79.0,117.0


Dropping outliers is not as straightforward as it may be in other types of analysis, since we are trying to understand when this spikes on pollution occur. Likewise, imputation for missing values is more than replacing `pd.NA`'s with the mean/median for every column, because we are dealing with a time-series. Data visualization will proceed, to understand the behaviour of data and apply the most adequate procedures for outliers and missing values 

In [14]:
with open('data/dataframe.pkl', 'wb') as file:
            pickle.dump(df, file)